In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
from datetime import datetime,timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loading_dir='/content/drive/MyDrive/Topic Mining Project/new_data/'

In [ ]:
TOKEN = '67b0372029c40988b5df783c6f4c2cb49f9f82a2'
HEADERS = {'Content-Type': 'application/json'}

requestResponse = requests.get(f"https://api.tiingo.com/tiingo/news?&limit=1000&token={TOKEN}", headers=HEADERS)
df=pd.DataFrame(requestResponse.json())
if type(df.iloc[0]['tickers'])==str:
  df=df[len(df['tickers'])!='[]']
elif type(df.iloc[0]['tickers'])==list:
  df=df[df['tickers'].apply(lambda x: len(x)>0)]
else:
  raise ValueError

df['publishedDate']=df['publishedDate'].apply(lambda x: x[:10])
df.drop_duplicates(subset='id',inplace=True)


df0=pd.read_csv(loading_dir+'news_data/news_with_ticker.csv',index_col=0)
index_set=set(df0['id'])
df=df[df['id'].apply(lambda x: x not in index_set)]
pd.concat([df0,df]).to_csv(loading_dir+'news_data/news_with_ticker.csv')

In [ ]:
pd.read_csv(loading_dir+'news_data/news_with_ticker.csv',index_col=0)

,id,publishedDate,title,url,description,source,tags,crawlDate,tickers
3724,61119206,2024-01-12,Novo Nordisk resumes shipments of Wegovy 1.7 m...,https://www.aol.com/news/novo-nordisk-resumes-...,"Along with lower strength doses of 0.25 mg, 0....",aol.com,"['Drug Shortage', 'Healthcare', 'Mg Dose', 'Mi...",2024-01-13T00:13:28.980481Z,['nvo']
3727,61119103,2024-01-12,ODOT suspended troubled logging truck company ...,https://www.oregonlive.com/crime/2024/01/odot-...,The state had documented 199 safety violations...,oregonlive.com,"['Industrials', 'Stock']",2024-01-13T00:06:28.318035Z,['cmi']
3731,61119052,2024-01-12,"ROSEN, LEADING INVESTOR COUNSEL, Encourages Ve...",https://www.benzinga.com/pressreleases/24/01/g...,"NEW YORK, Jan. 12, 2024 (GLOBE NEWSWIRE) -- WH...",benzinga.com,"['Legal', 'Mdrx', 'News', 'Press Releases', 'S...",2024-01-13T00:02:16.104075Z,"['mdrx', 'ne-ws']"
3732,61119078,2024-01-12,"Sustainability, AI and a lost race: How Formul...",https://www.independent.co.uk/sport/formula-e/...,With the race in India off the calendar it’s n...,independent.co.uk,"['Ai', 'Artificial Intelligence', 'Consumer Cy...",2024-01-13T00:04:32.800681Z,"['hmc', 'hndaf', 'nsany']"
3733,61120710,2024-01-12,"Sustainability, AI and a lost race: How Formul...",https://www.aol.com/news/sustainability-ai-los...,With the race in India off the calendar it’s n...,aol.com,"['Aggressive Growth Plans', 'Alberto Longo', '...",2024-01-13T01:08:42.988413Z,"['hmc', 'hndaf', 'nsany']"
...,...,...,...,...,...,...,...,...,...
985,64297092,2024-04-13,What's the average superannuation balance at r...,https://www.fool.com.au/2024/04/14/whats-the-a...,We investigate what retirement costs are and w...,fool.com.au,"['Australia', 'Financial Services', 'Stock', '...",2024-04-13T20:32:23.812886Z,"['anz', 'bhp', 'fmg', 'fsugy', 'wbc', 'wbk', '..."
988,64297134,2024-04-13,Donald Trump And Mike Johnson Are Pushing For ...,https://www.benzinga.com/news/24/04/38238164/d...,Former President Donald Trump and House Speake...,benzinga.com,"['2024 Election', 'Ai Generated', 'Benzinga', ...",2024-04-13T20:34:17.428398Z,['ne-ws']
992,64297188,2024-04-13,BetMGM NC bonus + bet365 NC promo code: Use $1...,https://www.newsweek.com/betmgm-nc-bonus-bet36...,Use our BetMGM NC bonus and bet365 NC promo co...,newsweek.com,"['Financial Services', 'Stock', 'Unknown Sector']",2024-04-13T20:41:27.593242Z,"['pypl', 'pyplv']"
994,64297361,2024-04-13,15 Industries with the Highest Electricity Con...,https://finance.yahoo.com/news/15-industries-h...,"In this article, we list and discuss the 15 In...",finance.yahoo.com,"['Electricity Consumption', 'Electricity Gener...",2024-04-13T21:01:35.209260Z,"['exc', 'excu', 'fe', 'nee']"


In [ ]:
from datetime import datetime,timedelta

start_date = datetime(2024, 1, 10)
current_date = datetime.now()
delta = current_date - start_date
date_list=[]
for i in range(delta.days + 1):
    date = start_date + timedelta(days=i)
    date_list.append(date.strftime("%Y-%m-%d"))

In [ ]:
TOKEN = '67b0372029c40988b5df783c6f4c2cb49f9f82a2'
HEADERS = {'Content-Type': 'application/json'}
data=pd.DataFrame()

for date,next_date in tqdm(zip(date_list[:-1],date_list[1:]),total=len(date_list)-1):
  requestResponse = requests.get(f"https://api.tiingo.com/tiingo/news?startDate={date}&endDate={next_date}&limit=10000&token={TOKEN}", headers=HEADERS)
  df=pd.DataFrame(requestResponse.json())
  if len(df)==0:
    continue
  if type(df.iloc[0]['tickers'])==str:
    df=df[len(df['tickers'])!='[]']
  elif type(df.iloc[0]['tickers'])==list:
    df=df[df['tickers'].apply(lambda x: len(x)>0)]
  else:
    raise ValueError

  df['publishedDate']=df['publishedDate'].apply(lambda x: x[:10])
  df.drop_duplicates(subset='id',inplace=True)
  df=df[df['publishedDate']==date]
  data=pd.concat([data,df])
data

 15%|█▌        | 15/98 [00:23<02:48,  2.03s/it]<ipython-input-7-28b662c6a69f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publishedDate']=df['publishedDate'].apply(lambda x: x[:10])
<ipython-input-7-28b662c6a69f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='id',inplace=True)
 16%|█▋        | 16/98 [00:24<02:40,  1.95s/it]<ipython-input-7-28b662c6a69f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,id,publishedDate,title,url,description,source,tags,crawlDate,tickers
32,61286153,2024-01-18,Does The Market Have A Low Tolerance For Domai...,https://simplywall.st/stocks/au/media/asx-dhg/...,Domain Holdings Australia (ASX:DHG) has had a ...,simplywall.st,"[Australia, Stock, Unknown Sector]",2024-01-19T00:15:17.215543Z,[dhg]
37,61285863,2024-01-18,Altcoins Forge Distinct Paths Amid Bitcoin Spo...,https://nulltx.com/altcoins-forge-distinct-pat...,Following the recent milestone of Bitcoin Spot...,nulltx.com,"[Altcoin, Altcoins, Bitcoin, Btc, Crypto, Cryp...",2024-01-18T23:59:32.077545Z,"[btc, ne-ws, pifi]"
38,61286136,2024-01-18,Korea’s Myrealtrip cashes in on travel rebound...,https://techcrunch.com/2024/01/18/koreas-myrea...,Myrealtrip is the latest travel tech company t...,techcrunch.com,"[Consumer Cyclical, South Korea, Startups, Sto...",2024-01-19T00:13:25.278477Z,"[expe, lexea, lexeb, sftbf]"
43,61285947,2024-01-18,Apple hired former ITC chair amid smartwatch p...,https://www.politico.com/newsletters/politico-...,"Polsinelli’s Deanna Okun, a trade lawyer who s...",politico.com,"[Healthcare, Stock, Technology, Unknown Sector]",2024-01-19T00:04:25.844718Z,"[masi, msft, pua]"
46,61287232,2024-01-18,Los Angeles Chargers interview former Titans c...,https://www.aol.com/news/los-angeles-chargers-...,Former Tennessee Titans coach Mike Vrabel was ...,aol.com,"[Atlanta Falcons, Baltimore Ravens, Brandon St...",2024-01-19T01:11:01.294580Z,[nfl]
...,...,...,...,...,...,...,...,...,...
9977,64375450,2024-04-16,"DEADLINE ALERT for INOD, DKS, FOXF, and LTRX: ...",https://www.benzinga.com/pressreleases/24/04/g...,"LOS ANGELES, April 16, 2024 (GLOBE NEWSWIRE) -...",benzinga.com,"[Consumer Cyclical, Dks, Foxf, Inod, Legal, Lt...",2024-04-16T16:13:39.417379Z,"[dks, foxf, inod, ltrx, ne-ws]"
9981,64375366,2024-04-16,Rob Schneider’s Comedy Is So Bad It’s Even Off...,https://www.vanityfair.com/hollywood/rob-schne...,"According to Politico, a set from the comedian...",vanityfair.com,"[Conservatives, Consumer Cyclical, Hollywood, ...",2024-04-16T16:11:33.020665Z,[nflx]
9986,64375605,2024-04-16,Canadian Dollar hits fresh five-month lows aft...,https://www.fxstreet.com/news/canadian-dollar-...,The Canadian Dollar (CAD) is going through a s...,fxstreet.com,"[Cpi, CryptoCurrency, Currencies, FX, Financia...",2024-04-16T16:16:16.805700Z,[ry]
9991,64375261,2024-04-16,Indus Towers wants Vodafone Idea to settle all...,https://www.livemint.com/industry/indus-towers...,Indus had collected ₹300 crore from VI on acc...,livemint.com,"[Communication Services, CryptoCurrency, Dues,...",2024-04-16T16:08:38.684389Z,"[idea, vod]"


In [ ]:
data.to_csv(loading_dir+'news_data/news_with_ticker.csv')

In [ ]:
data.head()

,id,publishedDate,title,url,description,source,tags,crawlDate,tickers
32,61286153,2024-01-18,Does The Market Have A Low Tolerance For Domai...,https://simplywall.st/stocks/au/media/asx-dhg/...,Domain Holdings Australia (ASX:DHG) has had a ...,simplywall.st,"[Australia, Stock, Unknown Sector]",2024-01-19T00:15:17.215543Z,[dhg]
37,61285863,2024-01-18,Altcoins Forge Distinct Paths Amid Bitcoin Spo...,https://nulltx.com/altcoins-forge-distinct-pat...,Following the recent milestone of Bitcoin Spot...,nulltx.com,"[Altcoin, Altcoins, Bitcoin, Btc, Crypto, Cryp...",2024-01-18T23:59:32.077545Z,"[btc, ne-ws, pifi]"
38,61286136,2024-01-18,Korea’s Myrealtrip cashes in on travel rebound...,https://techcrunch.com/2024/01/18/koreas-myrea...,Myrealtrip is the latest travel tech company t...,techcrunch.com,"[Consumer Cyclical, South Korea, Startups, Sto...",2024-01-19T00:13:25.278477Z,"[expe, lexea, lexeb, sftbf]"
43,61285947,2024-01-18,Apple hired former ITC chair amid smartwatch p...,https://www.politico.com/newsletters/politico-...,"Polsinelli’s Deanna Okun, a trade lawyer who s...",politico.com,"[Healthcare, Stock, Technology, Unknown Sector]",2024-01-19T00:04:25.844718Z,"[masi, msft, pua]"
46,61287232,2024-01-18,Los Angeles Chargers interview former Titans c...,https://www.aol.com/news/los-angeles-chargers-...,Former Tennessee Titans coach Mike Vrabel was ...,aol.com,"[Atlanta Falcons, Baltimore Ravens, Brandon St...",2024-01-19T01:11:01.294580Z,[nfl]


In [ ]:
data

,id,publishedDate,title,url,description,source,tags,crawlDate,tickers
3724,61119206,2024-01-12,Novo Nordisk resumes shipments of Wegovy 1.7 m...,https://www.aol.com/news/novo-nordisk-resumes-...,"Along with lower strength doses of 0.25 mg, 0....",aol.com,"[Drug Shortage, Healthcare, Mg Dose, Milligram...",2024-01-13T00:13:28.980481Z,[nvo]
3727,61119103,2024-01-12,ODOT suspended troubled logging truck company ...,https://www.oregonlive.com/crime/2024/01/odot-...,The state had documented 199 safety violations...,oregonlive.com,"[Industrials, Stock]",2024-01-13T00:06:28.318035Z,[cmi]
3731,61119052,2024-01-12,"ROSEN, LEADING INVESTOR COUNSEL, Encourages Ve...",https://www.benzinga.com/pressreleases/24/01/g...,"NEW YORK, Jan. 12, 2024 (GLOBE NEWSWIRE) -- WH...",benzinga.com,"[Legal, Mdrx, News, Press Releases, Stock, Tec...",2024-01-13T00:02:16.104075Z,"[mdrx, ne-ws]"
3732,61119078,2024-01-12,"Sustainability, AI and a lost race: How Formul...",https://www.independent.co.uk/sport/formula-e/...,With the race in India off the calendar it’s n...,independent.co.uk,"[Ai, Artificial Intelligence, Consumer Cyclica...",2024-01-13T00:04:32.800681Z,"[hmc, hndaf, nsany]"
3733,61120710,2024-01-12,"Sustainability, AI and a lost race: How Formul...",https://www.aol.com/news/sustainability-ai-los...,With the race in India off the calendar it’s n...,aol.com,"[Aggressive Growth Plans, Alberto Longo, Along...",2024-01-13T01:08:42.988413Z,"[hmc, hndaf, nsany]"
...,...,...,...,...,...,...,...,...,...
9983,64237267,2024-04-11,How to Use Android’s Find My Device Network,https://gizmodo.com/how-to-use-android-find-my...,"Like Apple’s AirTags, Android’s network only w...",gizmodo.com,"[Airtag, Android, Bluetooth, Cloud Clients, Co...",2024-04-11T15:48:50.417965Z,"[goog, googl, sony, tile]"
9988,64237115,2024-04-11,Church & Dwight to Webcast Discussion of First...,https://www.businesswire.com/news/home/2024041...,"Church & Dwight Co., Inc. (NYSE: CHD) will web...",businesswire.com,"[Consumer Defensive, Stock]",2024-04-11T15:46:27.540025Z,[chd]
9990,64246219,2024-04-11,Church & Dwight to Webcast Discussion of First...,http://www.businesswire.com/news/home/20240411...,"Church & Dwight Co., Inc. (NYSE: CHD) will web...",businesswire.com,"[Consumer Defensive, Stock]",2024-04-11T20:20:44.205644Z,[chd]
9992,64246184,2024-04-11,GROUPE SEB : FINALISATION DE L’ACQUISITION DE ...,http://www.businesswire.com/news/home/20240411...,Regulatory News: À la suite de la communicatio...,businesswire.com,"[Industrials, Stock]",2024-04-11T20:20:40.671145Z,[seb]
